# **추천시스템**
## **추천시스템 정리**
1. **내용 기반 추천시스템 : 객관적인 내용을** 기준으로 모델링
1. **협업필터링** : 영화/ 사용자별 선호도 정보를 활용하여 모델링
    1. **사용자/상품기반 협업필터링 : 동질의 사용자/ 상품정보를** 활용한 학습
    1. **잠재성요인 모델 협업필터링 : 전체 테이블로** 학습하여 $U, V$ 근사행렬 압축한 뒤, Broad Cast로 예측모델을 생성합니다
- 주요 추천 알고리즘 목록
<div><img src="./data/collab.png" width=500/></div>

<br/>
# **1 문서 분석 알고리즘**
[**(GitHub)**](https://github.com/your-first-ml-book/Examples) **처음 배우는 머신러닝** 11장 문서 분석 시스템 만들기
```
ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question
```
## **01 스팸 구분학습 모델링**
- Logistic Regression 학습모델을 사용합니다

### **1) 스팸 데이터 불러오기**
- 스팸/ 노스팸을 기준으로 **label을** 추가합니다
- **8,713 개의 token** 으로 구성된 학습목록을 생성합니다
- 활용시 **vocabluary** 목록과 **tf-idf** 로 변환 뒤 예측 합니다

In [1]:
# 정상 메일이면 0, 스펨이면 1 을 Label 합니다
# vocabulary : 인덱스별 단어목록
documents, labels = [], []
with open('data/SMSSpamCollection') as file_handle:
    for line in file_handle: 
        if line.startswith('spam\t'):
            labels.append(1)
            documents.append(line[len('spam\t'):])
        elif line.startswith('ham\t'):
            labels.append(0)
            documents.append(line[len('ham\t'):])

# 단어 빈도 피처 (idf 없으면 단어빈도(term frequency)가 생성)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
documents      = [doc.replace('\n','').lower() for doc in documents] # 줄바꿈,소문자 변환(차이는 없음)
vectorizer     = CountVectorizer()                   # 단어 횟수 피처
term_counts    = vectorizer.fit_transform(documents) # 문서내 단어횟수
vocabulary     = vectorizer.get_feature_names()      # 인덱스별 단어 List
tf_transformer = TfidfTransformer(use_idf=False).fit(term_counts)
features       = tf_transformer.transform(term_counts) # features.todense()
features.toarray().shape                             # label, token

# import collections
# collections.Counter(list(term_counts.toarray()[0])) # Counter Array
# collections.Counter(list(features.toarray()[0])) # tf-idf Array

(5574, 8713)

### **2) 회귀모델 학습**
**sklearn 모듈을** 활용한 **LogisticRegression** 학습 및 검증

In [2]:
# train, test 50% 분할 뒤 회귀모델을 만들고 검증합니다
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size = 0.5, random_state = 0)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# 검증 데이터 분류허가
print("train accuracy: {:.5f}\ntest accuracy : {:.5f}".format(
    classifier.score(X_train, y_train), classifier.score(X_test, y_test)))
X_train.toarray().shape

train accuracy: 0.96842
test accuracy : 0.96340


/home/markbaum/Python/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(2787, 8713)

In [3]:
# weights : 가중치 값, pairs : 가중치와 text
weights, pairs = classifier.coef_[0, :], []
for index, value in enumerate(weights):
    pairs.append((abs(value), vocabulary[index]))

pairs.sort(key=lambda x: x[0], reverse=True) # 분류기 점수정렬
print("전체단어 가중치 목록 갯수: ", len(pairs))
["score {:.4f} 단어: {}".format(par[0], par[1])  for par in pairs[:5]]

전체단어 가중치 목록 갯수:  8713


['score 4.2122 단어: call',
 'score 3.5507 단어: txt',
 'score 2.9129 단어: free',
 'score 2.8318 단어: stop',
 'score 2.6789 단어: text']

### **3) 학습된 회귀모델의 활용**
훈련당시 데이터와 동일한 환경으로 데이터를 변환하여 예측합니다

In [4]:
# 훈련데이터의 일부인 X_train.toarray()[1] 데이터로 스팸여부를 판단
text = X_train.toarray()[2]

import numpy as np
txt = np.argwhere(text!=0).T.tolist()[0]
txt = " ".join(list(map(lambda x : vocabulary[x], txt)))
print("Texts : {}\n스팸판단 : {}".format(
    txt, classifier.predict([text])))  # 판단결과 스펨에 해당합니다

Texts : again appt is the when yes
스팸판단 : [0]


In [5]:
# 임의의 Text를 사용하여 스팸여부를 판단

sample_txts = """While the press doesn’t like writing about it.
08006344447 1000 2000 call cash claim freefone gift great guaranteed live news now operator or speak to your.
nor do I need them to, I donate my yearly Presidential salary of $400,000.00 to 
different agencies throughout the year, this to Homeland Security. 
If I didn’t do it there would be hell to pay from the FAKE NEWS MEDIA!"""
sample_txts = sample_txts.lower()
sample_txts = sample_txts.replace('\n', '')
sample_txts = sample_txts.split('.')
sample_txts

['while the press doesn’t like writing about it',
 '08006344447 1000 2000 call cash claim freefone gift great guaranteed live news now operator or speak to your',
 'nor do i need them to, i donate my yearly presidential salary of $400,000',
 '00 to different agencies throughout the year, this to homeland security',
 ' if i didn’t do it there would be hell to pay from the fake news media!']

In [6]:
# feacture 변환
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer     = CountVectorizer()                     # 단어 횟수 피처
term_counts    = vectorizer.fit_transform(sample_txts) # 문서내 단어횟수
voca           = vectorizer.get_feature_names()        # 인덱스별 단어 List
tf_transformer = TfidfTransformer(use_idf=False).fit(term_counts)
features       = tf_transformer.transform(term_counts) # features.todense()
print(features.toarray().shape)                        # label, token               

(5, 56)


In [7]:
array_text = []
for sentence in features.toarray():
    sent_arr = [0] * len(vocabulary)     # 빈 array 생성
    for no, tf_t in enumerate(sentence): # 단어별 빈도값 추출
        if tf_t != 0:                    # 유효값이 발견시
            if voca[no] in vocabulary:   # 학습가능 단어 포함여부 확인
                sent_arr[vocabulary.index(voca[no])] = sentence[no]
    array_text.append(sent_arr)
array_text = np.array(array_text)

classifier.predict(array_text)

array([0, 1, 0, 0, 0])

## **02 LDA 모델을 활용한 주제어 분석**
- 위에서 추출한 스팸데이터를 활용하여 **LDA 알고리즘으로** 주제별 단어를 묶습니다
- 전형적인 **비지도 학습** 알고리즘 입니다

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)
vectorizer  = CountVectorizer(stop_words='english', max_features=2000)
term_counts = vectorizer.fit_transform(documents)
vocabulary  = vectorizer.get_feature_names()
            
# LDA는 단어출현 갯수로 동작하므로 CountVectorizer를 활용
from sklearn.decomposition import LatentDirichletAllocation
topic_model = LatentDirichletAllocation(n_components=10)
topic_model.fit(term_counts) # 토픽 모델을 학습

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [9]:
# 학습된 토픽들을 하나씩 출력합니다.
topics = topic_model.components_
for topic_id, weights in enumerate(topics):
    print('topic %d' % topic_id, end=': ')
    pairs = [(abs(value), vocabulary[term_id]) for term_id, value in enumerate(weights)]
    pairs.sort(key=lambda x: x[0], reverse=True)
    for pair in pairs[:10]: print(pair[1], end=',')
    print()

topic 0: ll,later,sorry,hi,think,know,im,need,yeah,gonna,
topic 1: don,home,know,just,come,buy,going,time,want,stuff,
topic 2: good,day,love,dear,hope,happy,night,just,like,want,
topic 3: ok,lor,like,wat,ask,dun,thk,ur,yup,home,
topic 4: ur,going,tomorrow,just,got,work,ll,class,doing,come,
topic 5: tell,ur,week,dont,just,life,oh,sorry,tone,txt,
topic 6: stop,txt,text,com,ur,www,new,win,service,msg,
topic 7: got,da,send,ur,pick,cash,right,message,phone,way,
topic 8: reply,text,just,free,yes,stop,mobile,live,claim,ringtone,
topic 9: gt,lt,free,prize,mobile,won,claim,urgent,send,guaranteed,


<br/>
# **2 내용기반 추천 알고리즘**
- [**(GitHub)**](https://github.com/your-first-ml-book/Examples) **처음 배우는 머신러닝** 12장 영화 추천 시스템 만들기
- **movie_info_li : 영화의 정보**, **movie_plot_li : 영화의 줄거리** 저장
```
original_title 	overview
0 	Toy Story 	Led by Woody, Andy's toys live happily in his ...
1 	Jumanji 	When siblings Judy and Peter discover an encha... ```

## **01 영화 자료 불러오기**
영화 제목과, 영화 줄거리 정보를 불러옵니다

In [10]:
import pandas as pd
movies = pd.read_csv('data/movies_metadata.csv', usecols=['original_title', 'overview', 'title'], low_memory=False)
movies = movies.dropna(axis=0)
print(movies.shape)

movie_plot_li = movies['overview']
movie_info_li = movies['title']
movies.head(3)

(44506, 3)


,original_title,overview,title
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,Jumanji,When siblings Judy and Peter discover an encha...,Jumanji
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpier Old Men


In [11]:
movies.columns

Index(['original_title', 'overview', 'title'], dtype='object')

## **02 영화 줄거리를 활용한 Cosin 유사도 모델**
- **[A-z]** 정규식 과 **WordNetLemmatizer** 을 활용하여 분석가능한 객체를 추출
- **Memory Error** 로 인해 전체가 아닌 일부만 학습을 진행합니다 
- 다른 PC에서 학습 후 **pickle** 로 적용, 활용하는 방법도 가능합니다

### **1) cosin 유사도를 사용한 모델링**

In [12]:
%%time
# 잘 전처리된 token을 추출 합니다
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl       = WordNetLemmatizer()
        self.tokenizer = RegexpTokenizer('(?u)[A-z]+')
    
    def __call__(self, doc):  # 클래스 호출시 마다 실행(Tf-idf Vector 호출)
        return([self.wnl.lemmatize(t) for t in self.tokenizer.tokenize(doc)])

# 사이킷런에 위에서 정의한 토크나이저를 입력으로 넣습니다.
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer   = TfidfVectorizer(min_df=3, tokenizer=LemmaTokenizer(), stop_words='english')
X            = vectorizer.fit_transform(movie_plot_li[:10000]) # 메모리 오류로 갯수를 제한
vocabluary   = vectorizer.get_feature_names()

# 비슷한 영화 추천하는 Cosin 유사모델 만들기
from sklearn.metrics.pairwise import cosine_similarity
movie_sim = cosine_similarity(X)
print(movie_sim.shape)

/home/markbaum/Python/python/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(10000, 10000)
CPU times: user 5.13 s, sys: 545 ms, total: 5.67 s
Wall time: 5.75 s


### **2) 학습 모델을 활용한 유사측정 영화목록 출력하기**
- **movie_sim** 유사도 측정 Matrix 를 사용하여, 인덱스별 유사도를 측정 가능합니다
- 위의 추출내용을 **reverse=True** 로 데이터를 정렬 한 뒤, 상위 객체들을 출력합니다

In [13]:
# 특정 영화와 유사한 영화목록 출력하기
def similar_recommend_by_movie_id(movielens_id, rank=8):
    movie_index    = movielens_id - 1
    similar_movies = sorted(list(enumerate(movie_sim[movie_index])),key=lambda x:x[1], reverse=True)
    # 유사도 측정결과를 출력
    print("----- {} : 관람객 추천영화 -------".format(movie_info_li[similar_movies[0][0]]))
    for no, movie_idx in enumerate(similar_movies[1:rank]):
        print('추천영화 %d순위 : %s'%(no, movie_info_li[movie_idx[0]]))
        
similar_recommend_by_movie_id(55)

----- Georgia : 관람객 추천영화 -------
추천영화 0순위 : Don't Look in the Basement
추천영화 1순위 : Sour Grapes
추천영화 2순위 : James and the Giant Peach
추천영화 3순위 : White Zombie
추천영화 4순위 : The Man Who Knew Too Little
추천영화 5순위 : Heaven's Burning
추천영화 6순위 : Made


## **03 영화 줄거리를 활용한 linear_kernel 유사도 모델**
위에서 구현되는 기능을 보다 간결하게 처리하는 방법을 구현합니다
- 도서 : **머신러닝 완벽 가이드 (2019/3)** [**GitHub**](https://github.com/wikibook/ml-definitive-guide)
- 개념들 보완 : 딥러닝 활용한 자연어 입문 [**(WikiBook)**](https://wikidocs.net/24603)

### **1) linear_kernel 유사도를 사용한 모델링**

In [14]:
# 5,000 개의 영화, 22,304 단어행렬 (stopword 제거)
from sklearn.feature_extraction.text import TfidfVectorizer
data             = movies.head(12000)             # 부하를 줄이기 위해 5000개만 추출
tfidf            = TfidfVectorizer(stop_words='english')
data['overview'] = data['overview'].fillna('') # 줄거리 NaN 면 인덱스 제거
tfidf_matrix     = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)                      # overview에 대해서 tf-idf 수행

/home/markbaum/Python/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(12000, 35540)


In [15]:
%%time
# TF-IDF Vectorizer간 Dot Product 계산 Score 제공
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices    = pd.Series(data.index, index=data['title']).drop_duplicates()
idx        = indices['Father of the Bride Part II']
print("\nFather of the Bride Part II 의 인덱스: ", idx)


Father of the Bride Part II 의 인덱스:  4
CPU times: user 815 ms, sys: 653 ms, total: 1.47 s
Wall time: 1.47 s


### **2) 학습한 모델을 활용하기**
linear_kernel 유사도 모델을 사용하여 결과를 출력합니다

In [16]:
# OverView 데이터를 사용하여 영화간 유사도를 측정합니다
def get_recommendations(idx, cosine_sim=cosine_sim, rank=11):
#     idx        = indices[title]                # 해당영화의 타이틀로 인덱스를 호출
    sim_scores = list(enumerate(cosine_sim[idx])) # 모든 영화에 대한 해당영화 유사도
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # 유사도 정렬
    sim_scores = sim_scores[0: rank]           # 가장 유사한 10개의 영화를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores] # 유사도 높은 10개 영화
    return data['title'].iloc[movie_indices]   # 가장 유사한 10개의 영화의 제목을 리턴

get_recommendations(0)

0                     Toy Story
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
5797              Class of 1984
Name: title, dtype: object

<br/>
# **3 협업 필터링**
- **협업 필터링을** 활용하면 **보다 적은 데이터로** 학습이 가능합니다
- **surprise, fastFM** 등의 모듈을 활용하면 보다 쉬운 접근이 가능합니다

## **01 fastFM 모듈을 사용한 예측 모델링**
- **surprise** 모델보다 **다양한 데이터 행렬을** 사용한 학습이 가능합니다
- **학습용 dataset과, label 데이터셋을** 구분한 뒤 **모델을 학습** 합니다

In [17]:
# 나이를 제외한 정보는 범주형으로 변환됩니다
from sklearn.feature_extraction import DictVectorizer
train = [
    {"user": "1", "item": "5", "age": 19},
    {"user": "2", "item": "43", "age": 33},
    {"user": "3", "item": "20", "age": 55},
    {"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()       # Key 피쳐값은 v.feature_names_ 저장
X = v.fit_transform(train) # age값, user BOW(4), item BOW(4)
print(v.feature_names_)
print(X.toarray())

['age', 'item=10', 'item=20', 'item=43', 'item=5', 'user=1', 'user=2', 'user=3', 'user=4']
[[19.  0.  0.  0.  1.  1.  0.  0.  0.]
 [33.  0.  0.  1.  0.  0.  1.  0.  0.]
 [55.  0.  1.  0.  0.  0.  0.  1.  0.]
 [20.  1.  0.  0.  0.  0.  0.  0.  1.]]


In [18]:
# 1~4 user 들의 평점(Target) 정보를 활용하여 회귀모델을 학습
import numpy as np
y  = np.array([5.0, 1.0, 2.0, 4.0]) 

from fastFM import als
fm = als.FMRegression(n_iter = 1000, 
                      init_stdev = 0.1, 
                      rank = 2, 
                      l2_reg_w = 0.1, 
                      l2_reg_V = 0.5)
fm.fit(X, y)
fm

FMRegression(init_stdev=0.1, l2_reg=0, l2_reg_V=0.5, l2_reg_w=0.1,
       n_iter=1000, random_state=123, rank=2)

In [19]:
# 새로운 사용자 정보를 입력하면 예측 3.6 평점을 출력 합니다
fm.predict(v.transform({"user": "5", "item": "10", "age": 24}))

array([3.60775939])

## **02 Surprise 모듈을 사용한 예측 모델링**
**협업 필터링은 itemID, userID, rating** 3개의 필드 값 정보로만 구성 됩니다

### **1) 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝**
- **K-Fold** 교차검증을 사용하여 **검증 알고리즘** 을 확인 합니다
- **GridSearchCV** 등을 사용하여 **최적의 파라미터를** 찾습니다

In [20]:
%%time
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate 
ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
reader  = Reader(rating_scale = (0.5, 5.0))  # Reader 인스턴스
data    = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo    = SVD(random_state = 0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8987  0.8956  0.8979  0.8953  0.8945  0.8964  0.0016  
MAE (testset)     0.6933  0.6914  0.6887  0.6880  0.6877  0.6898  0.0022  
Fit time          3.92    3.98    3.93    3.93    4.37    4.03    0.17    
Test time         0.14    0.13    0.13    0.21    0.13    0.15    0.03    
CPU times: user 21.8 s, sys: 76.9 ms, total: 21.9 s
Wall time: 22 s


In [21]:
%%time
# 최적화 검증용 파라미터, GridSearchCV (KFold CV 3개 분할)
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print("Bast Score: {}\nParams: {}".format(
    gs.best_score['rmse'], gs.best_params['rmse']))

Bast Score: 0.899165305444369
Params: {'n_epochs': 20, 'n_factors': 50}
CPU times: user 1min 5s, sys: 540 ms, total: 1min 6s
Wall time: 2min 4s


### **2) 최적의 파라미터를 사용한 SVD 회귀모델 학습**
- 위에서 사용한 **K-Fold** 와 **CrossValidation** 결과를 활용합니다
- **sklearn 모듈을** 활용한 **LogisticRegression** 학습 및 검증

In [22]:
%%time
import pandas as pd
from surprise import Reader, Dataset, SVD
# ratings = pd.read_csv('./data/ml-latest-small/ratings.csv') 
# ratings.head(3)

# UserID, MovieID, ratting 3개 필드만 학습
# reader  = Reader(rating_scale=(0.5, 5.0))
# data    = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo    = SVD(n_factors=50, n_epochs=20, random_state=0)

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo.fit(trainset)
algo

CPU times: user 2.54 s, sys: 15.9 ms, total: 2.56 s
Wall time: 2.56 s


In [23]:
predictions = algo.test(testset)
[(pred.uid, pred.iid, pred.est)  for pred in predictions[:3]]

[(30, 254, 3.7092901755922996),
 (652, 2626, 4.172573234363497),
 (466, 2174, 3.673029601193435)]

In [24]:
from surprise import accuracy 
print(accuracy.rmse(predictions))

# 특정한 사용자/ 영화의 유사도 측정
userid, movieid  = str(196), str(302)
pred = algo.predict(userid, movieid)
print(pred)
"예측평점 :", pred.est

RMSE: 0.8908
0.8907754769926038
user: 196        item: 302        r_ui = None   est = 3.54   {'was_impossible': False}


('예측평점 :', 3.541978320867165)

## **03 잠재요인 협업 필터링**
### **1) 경사 하강법을 활용한 행렬분해**
- **user/ item based table** 을 예측하는데 있어서 $U, V$ 2개의 압축합니다
- 그리고 $U, V$ **행렬의 곱을** 활용하여 **비어있는 값들을 예측하는 테이블을** 생성합니다

In [25]:
# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화
# K-Fold 교차 잠재요인의 K 를 3 으로 설정 
import numpy as np
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

# P와 Q 매트릭스를 지정하고 정규분포의 random 값으로 채웁니다
num_users, num_items = R.shape
K = 3
np.random.seed(1234)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [26]:
from sklearn.metrics import mean_squared_error
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)  # 행렬 P와 Q.T의 내적으로 예측 R 생성    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros    = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse  = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    return rmse

In [27]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [(i, j, R[i,j])  for i in range(num_users) 
                             for j in range(num_items) 
                             if R[i,j] > 0 ]
steps, learning_rate, r_lambda = 1001, 0.01, 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # Regularization을 반영한 SGD 업데이트 공식
        eij    = r - np.dot(P[i, :], Q[j, :].T) # 실제 값과 예측 값의 차이
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 200) == 0 :
        print("# 반복학습 Step : {:5,} RMSE : {:.4f}".format(step, rmse))

# 반복학습 Step :     0 RMSE : 3.2843
# 반복학습 Step :   200 RMSE : 0.0218
# 반복학습 Step :   400 RMSE : 0.0146
# 반복학습 Step :   600 RMSE : 0.0144
# 반복학습 Step :   800 RMSE : 0.0144
# 반복학습 Step : 1,000 RMSE : 0.0144


In [28]:
pred_matrix = np.dot(P, Q.T)
print('학습전 행렬: \n{}\n\n경사하강 예측 행렬: \n{}'.format(
    R, np.round(pred_matrix, 3)))

학습전 행렬: 
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]

경사하강 예측 행렬: 
[[3.992 1.68  1.197 1.998 1.458]
 [5.41  4.976 0.659 2.987 1.005]
 [5.296 2.325 2.988 3.98  3.985]
 [4.971 2.005 1.004 2.004 1.095]]


### **2) 영화 리뷰 데이터를 활용한 행렬분해**
리뷰를 사용한 예측행렬을 생성합니다

In [29]:
import numpy as np
from sklearn.metrics import mean_squared_error

# RMSE 결과예측 함수를 정의합니다
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T) # P와 Q.T의 내적 곱 : 예측 R 행렬
    # 실제 R 행렬에서 NaN 아닌 값만 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros    = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse  = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)    
    return rmse

In [30]:
# 행렬 분해 후, 압축행렬의 곱으로 결과를 생성합니다
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    break_count = 0
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ] 
    for step in range(steps):      # SGD기법으로 P와 Q 매트릭스를 계속 업데이트
        for i, j, r in non_zeros:  # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 20) == 0 :
            print("# 학습의 반복: {:3} rmse: {:.4f}".format(step, rmse))
    return P, Q

In [31]:
%%time
import numpy as np
import pandas as pd
movies  = pd.read_csv('./data/ml-latest-small/movies.csv')
ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
rating_movies  = pd.merge(ratings, movies, on='movieId') # title 컬럼을 movies 와 Join
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# 압축행렬을 생성한뒤, 행렬곱을 사용하여 비어있는 값 들을 채웁니다
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=100, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

# 학습의 반복:   0 rmse: 2.9321
# 학습의 반복:  20 rmse: 0.5275
# 학습의 반복:  40 rmse: 0.3017
# 학습의 반복:  60 rmse: 0.2261
# 학습의 반복:  80 rmse: 0.1943
CPU times: user 2min 43s, sys: 21.7 s, total: 3min 5s
Wall time: 2min 35s


In [32]:
# 학습된 행렬을 사용하여 Pivot Table을 출력합니다
ratings_pred_matrix = pd.DataFrame(data = pred_matrix, 
                                   index = ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,1.745341,2.928824,1.150022,0.317918,1.286991,1.871280,1.846075,2.646222,2.902585,2.661586,...,2.466481,0.665347,2.536329,1.512936,2.512019,2.210895,0.533123,2.700853,2.361938,2.139596
2,2.162452,3.686570,1.542740,0.318964,1.740465,2.584770,2.545283,3.196354,3.670584,2.873576,...,3.374175,0.777538,2.977572,1.327954,3.101721,3.230171,0.632209,2.661355,2.884804,2.845496
3,2.250018,3.652691,1.543458,0.380940,1.689739,2.633695,2.563878,3.315305,4.030388,3.184506,...,3.468197,1.087605,3.537009,3.230556,3.345892,1.413009,0.627127,2.872369,3.217418,2.857210
